Stephen Jasper - 2301869853

##Import Library

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.decomposition import PCA
import timeit

##Load data

In [2]:
df = pd.read_csv('rawdata.csv')
print(df.head())

  SAMPLE_ID  snp_0  snp_1  snp_2  ...  snp_9998  snp_9999  snp_10000  STATUS
0    HCB181      1      0      0  ...         2         1          2       1
1    HCB182      1      0      0  ...         2         2          2       2
2    HCB183      1      0      0  ...         2         1          2       2
3    HCB184      1      0      0  ...         2         1          2       2
4    HCB185      1      0      0  ...         2         2          2       1

[5 rows x 10003 columns]


**Separate between feature data and label data**

In [3]:
feature = df.iloc[:, 1:-1]
label = df['STATUS'].values.reshape(-1,1)

##Split the data to train, test, validate data

In [4]:
x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.2)

In [5]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2)

**Pre-processing the y data**

In [6]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

#Convert y data to array. Because before y data is spars matrix, and it is cannot be use in model
y_train = encoder.fit_transform(y_train).toarray()
y_validate = encoder.transform(y_validate).toarray()
y_test = encoder.transform(y_test).toarray()

**Reshape into 3D for CNN model**

In [7]:
input_shape = (x_train.shape[1], 1)

x_train = x_train.values.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.values.reshape(x_test.shape[0], x_test.shape[1], 1)
x_validate = x_validate.values.reshape(x_validate.shape[0], x_validate.shape[1], 1)

print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')
print(f'x_validate shape: {x_validate.shape}')

x_train shape: (56, 10001, 1)
x_test shape: (18, 10001, 1)
x_validate shape: (15, 10001, 1)


##Implement for CNN model

**Build the CNN model using 1D conv**

In [8]:
CNN_raw_model = Sequential()
CNN_raw_model.add(Conv1D(32, kernel_size=(3), activation='relu', input_shape=input_shape))
#32 filter, with 1 filter of size 3, because 1D
CNN_raw_model.add(Dropout(0.2))
CNN_raw_model.add(Conv1D(64, kernel_size=(3), activation='relu'))
#64 filter, with 1 filter of size 3
CNN_raw_model.add(MaxPooling1D(pool_size=(2)))
#Pooling to reduce dimensionality from featured map and avoid of overfitting.
CNN_raw_model.add(Dropout(0.3))
CNN_raw_model.add(Flatten())
#Flatten is a proses to change the data from 2D to vector
#dense = fully connected layer
CNN_raw_model.add(Dense(64, activation='relu'))
CNN_raw_model.add(Dense(128, activation='relu'))
CNN_raw_model.add(Dense(2, activation='softmax'))

In [9]:
CNN_raw_model.compile(optimizer= Adam(lr= 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


**Train the CNN model**

In [10]:
start1 = timeit.default_timer()
history_2 = CNN_raw_model.fit(x_train, y_train, batch_size= 300, epochs=200, validation_data=(x_validate, y_validate))
end1 = timeit.default_timer()

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 0.7304 - accuracy: 0.4821 - val_loss: 24.0101 - val_accuracy: 0.5333
Epoch 2/200
1/1 [==============================] - 0s 74ms/step - loss: 33.1206 - accuracy: 0.5179 - val_loss: 3.9782 - val_accuracy: 0.5333
Epoch 3/200
1/1 [==============================] - 0s 74ms/step - loss: 5.1937 - accuracy: 0.5179 - val_loss: 2.5606 - val_accuracy: 0.4667
Epoch 4/200
1/1 [==============================] - 0s 72ms/step - loss: 3.4345 - accuracy: 0.4821 - val_loss: 1.5508 - val_accuracy: 0.4667
Epoch 5/200
1/1 [==============================] - 0s 71ms/step - loss: 2.0597 - accuracy: 0.4821 - val_loss: 0.7267 - val_accuracy: 0.4667
Epoch 6/200
1/1 [==============================] - 0s 76ms/step - loss: 0.7269 - accuracy: 0.4821 - val_loss: 1.0988 - val_accuracy: 0.5333
Epoch 7/200
1/1 [==============================] - 0s 69ms/step - loss: 1.5841 - accuracy: 0.5179 - val_loss: 0.9295 - val_accuracy: 0.5333
Epoch 8/200
1/1 [===

##Test the accuracy for CNN model

In [11]:
acc = CNN_raw_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy score for CNN -> DNA')
print('Test accuracy:', acc[1]*100)
print(f'Time without PCA -> {end1 - start1}')

Accuracy score for CNN -> DNA
Test accuracy: 50.0
Time without PCA -> 18.772414570000024


##Using PCA to reduce the dimentions

In [12]:
input_shape = (x_train.shape[1], 1)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1])
x_validate = x_validate.reshape(x_validate.shape[0], x_validate.shape[1])

print('x_train shape: {}'.format(x_train.shape))
print('x_test shape: {}'.format(x_test.shape))
print('x_validate shape: {}'.format(x_validate.shape))

x_train shape: (56, 10001)
x_test shape: (18, 10001)
x_validate shape: (15, 10001)


##PCA Model

In [13]:
pca = PCA(n_components = 30) #reduce componen into 30 feature
pca_x_train = pca.fit_transform(x_train)
pca_x_test = pca.transform(x_test)
pca_x_validate = pca.transform(x_validate)

#Inverse is used to make the data more clearly from PCA
pca_x_train = pca.inverse_transform(pca_x_train)
pca_x_test = pca.inverse_transform(pca_x_test)
pca_x_validate = pca.inverse_transform(pca_x_validate)

**Reshape into 3D again to use in CNN model**

In [14]:
x_train = pca_x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = pca_x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
x_validate = pca_x_validate.reshape(x_validate.shape[0], x_validate.shape[1], 1)

print('x_train shape: {}'.format(x_train.shape))
print('x_test shape: {}'.format(x_test.shape))
print('x_validate shape: {}'.format(x_validate.shape))

x_train shape: (56, 10001, 1)
x_test shape: (18, 10001, 1)
x_validate shape: (15, 10001, 1)


##Implement CNN + PCA model

**Build the CNN model that already using PCA**

In [15]:
CNN_raw_PCAmodel = Sequential()
CNN_raw_PCAmodel.add(Conv1D(32, kernel_size=(3), activation='relu', input_shape=input_shape))
CNN_raw_PCAmodel.add(Dropout(0.2))
CNN_raw_PCAmodel.add(Conv1D(64, kernel_size=(3), activation='relu'))
CNN_raw_PCAmodel.add(MaxPooling1D(pool_size=(2)))
CNN_raw_PCAmodel.add(Dropout(0.3))
CNN_raw_PCAmodel.add(Flatten())
CNN_raw_PCAmodel.add(Dense(64, activation='relu'))
CNN_raw_PCAmodel.add(Dense(128, activation='relu'))
CNN_raw_PCAmodel.add(Dense(2, activation='softmax'))

In [16]:
CNN_raw_PCAmodel.compile(optimizer= Adam(lr= 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


**Train the model**

In [17]:
start2 =timeit.default_timer()
history = CNN_raw_PCAmodel.fit(x_train, y_train, batch_size= 300, epochs=200, validation_data=(x_validate, y_validate))
end2 = timeit.default_timer()

Epoch 1/200
1/1 [==============================] - 1s 713ms/step - loss: 0.6991 - accuracy: 0.5357 - val_loss: 11.8353 - val_accuracy: 0.5333
Epoch 2/200
1/1 [==============================] - 0s 71ms/step - loss: 15.6984 - accuracy: 0.5179 - val_loss: 14.6909 - val_accuracy: 0.4667
Epoch 3/200
1/1 [==============================] - 0s 75ms/step - loss: 19.2569 - accuracy: 0.4821 - val_loss: 0.7650 - val_accuracy: 0.5333
Epoch 4/200
1/1 [==============================] - 0s 72ms/step - loss: 0.9214 - accuracy: 0.5179 - val_loss: 2.4828 - val_accuracy: 0.4667
Epoch 5/200
1/1 [==============================] - 0s 72ms/step - loss: 3.2076 - accuracy: 0.4821 - val_loss: 4.4399 - val_accuracy: 0.5333
Epoch 6/200
1/1 [==============================] - 0s 74ms/step - loss: 6.5135 - accuracy: 0.5179 - val_loss: 3.6399 - val_accuracy: 0.5333
Epoch 7/200
1/1 [==============================] - 0s 71ms/step - loss: 5.5031 - accuracy: 0.5179 - val_loss: 0.6943 - val_accuracy: 0.4667
Epoch 8/200
1/1

##Test the accuracy for CNN + PCA model

In [18]:
accPCA = CNN_raw_PCAmodel.evaluate(x_test, y_test, verbose=0)
print('Accuracy score for CNN + PCA -> DNA')
print('Test accuracy:', accPCA[1]*100)
print(f'Time with PCA -> {end2 - start2}')

Accuracy score for CNN + PCA -> DNA
Test accuracy: 55.55555820465088
Time with PCA -> 16.476340273000005
